In [3]:
ofns_desc_values_path = './datasets/column_values_for_clearing/ofns_desc_values.txt'
prem_typ_values_path = './datasets/column_values_for_clearing/prem_typ_values.txt'
with open(prem_typ_values_path) as f:
    columns = f.read().split('\n')
print(len(columns))

94


In [4]:
columns = {i: column for i,column in enumerate(columns)}
columns1 = columns.copy()

In [5]:
#Universal Sentence Encoder
import tensorflow as tf
from tensorflow_hub import load
import tensorflow_text
import numpy as np

In [6]:
use_model = load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')
# Define a function to get the embeddings of the product names
def get_embeddings(text_dict):#texts = {ID:text} 
    texts = list(text_dict.values())
    ids = list(text_dict.keys())
    # Create a tf.data.Dataset to load the texts in a memory-efficient way
    dataset = tf.data.Dataset.from_tensor_slices(texts)
    # Batch the dataset to reduce the memory required for processing the embeddings
    dataset = dataset.batch(2048)
    # Get the embeddings using the Universal Sentence Encoder
    embeddings = []
    for batch in texts:
        batch_embeddings = use_model(batch)
        # Convert the embeddings to float16 to reduce their size in memory
        batch_embeddings = tf.cast(batch_embeddings, tf.float16)
        embeddings.append(batch_embeddings)
    embeddings = np.concatenate(embeddings)
    embeddings_dict = {}
    for i, embedding in enumerate(embeddings):
        embeddings_dict[ids[i]] = embedding
        
    return embeddings_dict

In [7]:
# Get the embeddings of the product names in the databases
database1_embeddings = get_embeddings(columns)
database2_embeddings = get_embeddings(columns1)

# Compute the cosine similarity matrix between the product name embeddings
database1_ids, database1_embeddings = zip(*database1_embeddings.items())
database2_ids, database2_embeddings = zip(*database2_embeddings.items())

cosine_similarities = np.matmul(database1_embeddings, np.transpose(database2_embeddings))

# Find the most similar product names
max_similarities = np.max(cosine_similarities, axis=1)
max_indices = np.argmax(cosine_similarities, axis=1)

In [16]:
threshold = 0.62#strict >= 0.84, confident > 0.75, almost all > 0.52

def getTopMatches(top):
    global WantList
    WantList = True

    top_indices = np.argsort(cosine_similarities, axis=1)[:, ::-1][:, :top]
    top_similarities = np.sort(cosine_similarities, axis=1)[:, ::-1][:, :top]
    matched_products = {}
    
    for i, id1 in enumerate(database1_ids):
        id2_list = [database2_ids[j] for j in top_indices[i] if top_similarities[i][list(top_indices[i]).index(j)] >= threshold]
        matched_products[id1] = id2_list

    return matched_products

In [17]:
top_matches = getTopMatches(top=5)
for key in top_matches:
    print(columns[key], [columns[k] for k in top_matches[key] if k != key])

ABANDONED BUILDING ['PUBLIC BUILDING', 'COMMERCIAL BUILDING', 'BUS TERMINAL', 'DAYCARE FACILITY']
AIRPORT TERMINAL []
ATM []
BANK []
BAR/NIGHT CLUB ['SOCIAL CLUB/POLICY']
BEAUTY & NAIL SALON ['BEAUTY/NAIL SALON']
BEAUTY/NAIL SALON ['BEAUTY & NAIL SALON']
BOOK/CARD []
BRIDGE ['STREET', 'CEMETERY', 'BUS TERMINAL', 'SHOE']
BUS (NYC TRANSIT) ['BUS (OTHER)', 'TRANSIT FACILITY (OTHER)', 'BUS STOP']
BUS (OTHER) ['TRANSIT FACILITY (OTHER)', 'OTHER', 'BUS (NYC TRANSIT)', 'OTHER HOUSE OF WORSHIP']
BUS STOP ['BUS TERMINAL', 'BUS (NYC TRANSIT)']
BUS TERMINAL ['BUS STOP', 'STREET', 'REAL ESTATE', 'DEPARTMENT STORE']
CANDY STORE []
CEMETERY ['STREET', 'BRIDGE', 'SHOE', 'JEWELRY']
CHAIN STORE ['VARIETY STORE', 'SHOE STORE', 'DEPT STORE', 'JEWELRY STORE']
CHECK CASH ['CHECK CASHING BUSINESS', 'CHAIN STORE']
CHECK CASHING BUSINESS ['CHECK CASH']
CHURCH ['JEWELRY', 'CEMETERY', 'STREET']
CLOTHING BOUTIQUE ['CLOTHING/BOUTIQUE', 'SHOE STORE', 'SHOE']
CLOTHING/BOUTIQUE ['CLOTHING BOUTIQUE', 'SHOE STORE']
CO

In [ ]:
# prem_typ_desc
# changing to -> 
'BEAUTY & NAIL SALON','BEAUTY/NAIL SALON'
'CHECK CASHING BUSINESS','CHECK CASH'
'CLOTHING/BOUTIQUE', 'CLOTHING BOUTIQUE'
'COMMERCIAL BLDG', 'COMMERCIAL BUILDING'
'DEPARTMENT STORE', 'DEPT STORE'
'DOCTOR/DENTIST OFFICE','DOCTOR/DENTIST'
'FOOD SUPERMARKET','SUPERMARKET'
'JEWELRY STORE','JEWELRY'
'NONE',None
'(null)',None
'SHOE STORE','SHOE'
'SOCIAL CLUB/POLICY LOCATI','SOCIAL CLUB/POLICY'

In [ ]:
# ofns_desc
# changing to -> 
'KIDNAPPING AND RELATED OFFENSES', 'KIDNAPPING & RELATED OFFENSES'
'OTHER STATE LAWS (NON PENAL LA', 'OTHER STATE LAWS (NON PENAL LAW)'
'LOITERING/DEVIATE SEX', 'SEX CRIMES'
'FELONY SEX CRIMES', 'SEX CRIMES'
'ADMINISTRATIVE CODE', 'ADMINISTRATIVE CODES'
'INTOXICATED & IMPAIRED DRIVING','INTOXICATED/IMPAIRED DRIVING'
'NYS LAWS-UNCLASSIFIED VIOLATION', 'NYS LAWS-UNCLASSIFIED FELONY'
'ANTICIPATORY OFFENSES','OFFENSES AGAINST THE PERSON'
'CARDS','LOITERING/GAMBLING'
'LOITERING/GAMBLING', 'GAMBLING'
'(null)',None
'FRAUDULENT ACCOSTING','FRAUDS'
'OFFENSES INVOLVING FRAUD', 'FRAUDS'
# 'ROBBERY','BURGLARY' -- different
# Examples:
# Robbery: A person holding up a bank teller with a weapon and demanding money.
# Burglary: A person breaking into a house while the occupants are away to steal valuables.